In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
drive_path = "/content/drive/MyDrive/dat1"

In [3]:
import os
import numpy as np
import cv2
from glob import glob
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import tensorflow as tf
from tensorflow.keras.layers import  Conv2D, MaxPool2D, BatchNormalization, Activation, Input , Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, load_model

In [26]:

batch_size = 2
learning_rate = 1e-4
epochs = 40
height = 720
width = 1280

In [16]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [17]:
dataset_path = os.path.join("/content/drive/MyDrive/dat2", "augmented")
files_dir = os.path.join(drive_path, 'Colab Notebooks',"files", "augmented")
model_file= os.path.join(files_dir,"pentagon.h5")
log_file = os.path.join(files_dir,"loges-original.csv")

In [18]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path,"train","images","*")))
    train_y = sorted(glob(os.path.join(path,"train","masks","*")))

    valid_x = sorted(glob(os.path.join(path,"valid","images","*")))
    valid_y = sorted(glob(os.path.join(path,"valid","masks","*")))

    return (train_x,train_y) , (valid_x,valid_y)

#reading the image
def read_image(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = x/255.0
    return x

#reading the mask
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x


In [19]:
#tf data pipeline

def tf_parse(x,y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

def tf_dataset(x, y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [20]:
model = load_model(model_file)

In [21]:
model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 720, 1280, 3)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 720, 1280, 64)        1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 720, 1280, 64)        256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 720, 1280, 64)        0         ['batch_normalization[0][0]

In [27]:
(train_x,train_y) , (valid_x,valid_y) = load_data(dataset_path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [28]:
opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [29]:
callbacks = [

    ModelCheckpoint(model_file,verbose=1,save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss",factor=0.1,patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss',patience=20, restore_best_weights= False)
]

In [30]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs = epochs,
    callbacks=callbacks
)

Epoch 1/40
984/984 [==============================] - ETA: 0s - loss: 0.1310 - acc: 0.9474
Epoch 1: val_loss improved from inf to 0.30434, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/files/augmented/pentagon.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


984/984 [==============================] - 523s 499ms/step - loss: 0.1310 - acc: 0.9474 - val_loss: 0.3043 - val_acc: 0.9030 - lr: 1.0000e-04
Epoch 2/40
984/984 [==============================] - ETA: 0s - loss: 0.1027 - acc: 0.9586
Epoch 2: val_loss did not improve from 0.30434
984/984 [==============================] - 471s 478ms/step - loss: 0.1027 - acc: 0.9586 - val_loss: 0.3271 - val_acc: 0.9114 - lr: 1.0000e-04
Epoch 3/40
984/984 [==============================] - ETA: 0s - loss: 0.0897 - acc: 0.9646
Epoch 3: val_loss improved from 0.30434 to 0.21008, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/files/augmented/pentagon.h5
984/984 [==============================] - 475s 483ms/step - loss: 0.0897 - acc: 0.9646 - val_loss: 0.2101 - val_acc: 0.9142 - lr: 1.0000e-04
Epoch 4/40
984/984 [==============================] - ETA: 0s - loss: 0.0855 - acc: 0.9657
Epoch 4: val_loss improved from 0.21008 to 0.10518, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/fi

In [31]:
print("hej")

hej
